In [1]:
import cv2 as cv
import sys 

import sklearn
import numpy as np 
import os
import pandas as pd
import time

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
from sklearn.datasets import fetch_openml
import matplotlib as mpl
import matplotlib.pyplot as plt

from IPython.utils import io

In [2]:
def procces_image(img_path):
    img = cv.imread(img_path ,0)
    img = cv.resize(img , (100,100))
    img = img.reshape(100*100)
    return img

In [3]:
def load_data(path):
    x = []
    y = []

    for file in os.listdir(path):
        label = file.split('.')[-3]
        data = procces_image("{}/{}".format(path,file))
        x.append(data)
        y.append(label)
    return x,y

In [4]:
X,Y = load_data('data/Cats and dogs/train')

In [5]:
X = np.asarray(X)
Y = np.asarray(Y)

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_validate,y_train,y_validate = train_test_split(X,Y ,random_state=42)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.decomposition import PCA

preprocess = Pipeline([
    ('scaler',StandardScaler(with_std=False)),
    ('pca',PCA(n_components=0.95)),
    ('shuffle' , shuffle(random_state=42))
])

In [8]:
x_reduced = preprocess.fit_transform(x_train)
x_recovered = preprocess.inverse_transform(x_reduced)
x_validate_reduced = preprocess.transform(x_validate)

In [9]:
import xgboost as xgb

xgb_clf = xgb.XGBClassifier(eta=0.3,max_depth=10)
xgb_clf.fit(x_reduced,y_train)
xgb_clf.score(x_validate_reduced,y_validate)

[16:20:01] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.62656

In [10]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rf_clf.fit(x_reduced,y_train)
rf_clf.score(x_validate_reduced,y_validate)


0.5816

In [11]:
from sklearn.naive_bayes import GaussianNB


gnb_clf =  GaussianNB()

gnb_clf.fit(x_reduced,y_train)
gnb_clf.score(x_validate_reduced,y_validate)

0.55376

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

ada_clf =  AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=200,learning_rate=0.2)

ada_clf.fit(x_reduced,y_train)
ada_clf.score(x_validate_reduced,y_validate)

0.63328